## Hello! This notebook will be used for my capstone project.

In [3]:
import pandas as pd
import numpy as np

In [4]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


## INTRODUCTION

According to the car accident data, we will seek to predict the severity of an eventual collision in some place according to certain characteristics, in order to place warning signs for all drivers who pass through those areas.

## DATA

The data that will be used will be: the type of street (ADDRTYPE), the track conditions (ROADCON), the brightness of the place (LIGHTCOND) and the severity of the incident (SEVERITYCODE). To predict the severity of a place, the decision tree will be used. For which we will use 4 weather conditions with the highest number of incidents, 4 types of brightness in the area with the highest number of incidents, we will clean the data by eliminating the rows with null data and other data that we will not need at work. It will seek to catalog the areas according to the severity of accidents and to place warning signs to slow down.